# Dynatrace MCP Server を AgentCore Gateway に統合

## 概要

数千の開発チームがあり、それぞれが複数の可観測性ツールとモニタリングシステムを運用している大企業では、機能の管理と発見が重要な課題となります：

* **ツールの発見と共有**: チームは組織全体で可観測性ツールを発見し利用することに苦労しています。各チームが別々のゲートウェイを維持すべきでしょうか？運用オーバーヘッドを発生させずに、ゲートウェイ URL を共有し、中央レジストリを管理するにはどうすればよいでしょうか？
* **ゲートウェイ管理**: 各可観測性システムごとに別々のゲートウェイを維持することは、スケール時にすぐに管理不能になります。
* **認証の複雑さ**: 複数のモニタリングプラットフォーム間で認証と認可を管理することは、特に機密性の高いエンタープライズデータでは、ますます複雑になります。
* **保守オーバーヘッド**: MCP 仕様の更新に追従するには、すべての実装で継続的な再作業とテストが必要です。

AgentCore Gateway は、Lambda や OpenAPI ツールに加えて、既存の MCP サーバー実装をターゲットとしてオンボーディングすることをサポートするようになりました。このチュートリアルでは、Dynatrace の MCP サーバーを AgentCore Gateway と統合して、可観測性機能への一元的なアクセスを提供する方法をデモンストレーションします。

![Diagram](images/dynatrace-mcp-server-target.png)

AgentCore Gateway が Dynatrace MCP サーバーをターゲットとして統合されたら、統合された MCP サーバーを介してツールを検索し、次に Strands Agent を使用してツールの一覧取得とツール呼び出し機能をデモンストレーションします。これらの各フローの詳細は以下で確認できます：


### チュートリアル詳細


| 情報                  | 詳細                                                      |
|:---------------------|:----------------------------------------------------------|
| チュートリアルタイプ   | インタラクティブ                                           |
| AgentCore コンポーネント | AgentCore Gateway、AgentCore Identity                    |
| エージェントフレームワーク | Strands Agents                                          |
| Gateway ターゲットタイプ | MCP サーバー                                             |
| エージェント           | Strands                                                  |
| Inbound 認証 IdP      | Amazon Cognito                                            |
| Outbound 認証         | OAuth2                                                    |
| LLM モデル            | Anthropic Claude Sonnet 4                                 |
| チュートリアル構成要素 | AgentCore Gateway の作成と AgentCore Gateway の呼び出し   |
| チュートリアルの業種   | 可観測性                                                  |
| 例の複雑さ            | 簡単                                                      |
| 使用 SDK              | boto3                                                     |

## チュートリアルアーキテクチャ

このチュートリアルは、より広範なエンタープライズの課題の実践的な例として機能します：**既存の可観測性 MCP サーバーを集中型 Gateway アーキテクチャに統合する方法。**

## 前提条件

このチュートリアルを実行するには、以下が必要です：
* Jupyter notebook（Python カーネル）
* AWS 認証情報
* Amazon Cognito
* [MCP サーバーエンドポイント](https://docs.dynatrace.com/docs/discover-dynatrace/platform/davis-ai/dynatrace-mcp)を持つ Dynatrace 環境
* Dynatrace で[OAuth クライアントを作成](https://docs.dynatrace.com/docs/manage/identity-access-management/access-tokens-and-oauth-clients/oauth-clients)

In [ ]:
# requirements ファイルからインストール
!pip install --force-reinstall -U -r requirements.txt --quiet


In [ ]:
import os
# SageMaker ノートブックを使用していない場合は AWS 認証情報を設定
# os.environ['AWS_ACCESS_KEY_ID'] = '' # アクセスキーを設定
# os.environ['AWS_SECRET_ACCESS_KEY'] = '' # シークレットキーを設定
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'us-east-1')

In [ ]:
# utils をインポート
import os
import sys

# 現在のスクリプトのディレクトリを取得
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # __file__ が定義されていない場合のフォールバック（例: Jupyter）

# utils.py を含むディレクトリに移動（1階層上）
utils_dir = os.path.abspath(os.path.join(current_dir, '..'))

# sys.path に追加
sys.path.insert(0, utils_dir)

# これで utils をインポートできます
import utils

# ロギングをセットアップ
import logging

# ノートブック環境用にロギングを設定
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

# 特定のロガーレベルを設定
logging.getLogger("strands").setLevel(logging.INFO)

## 設定

Dynatrace MCP Server の詳細を設定します：

In [ ]:
# Dynatrace MCP Server 設定
# $TENANT_ID を自分の ID に置き換えてください。Dynatrace 環境の URL で確認できます（例: https://abc12345.apps.dynatrace.com/）
DYNATRACE_MCP_SERVER_URL = "https://$TENANT_ID.apps.dynatrace.com/platform-reserved/mcp-gateway/v0.1/servers/dynatrace-mcp/mcp"
# OAUTH_CLIENT_ID（文字列、例: dt0s02.SAMPLE）- Dynatrace OAuth クライアント ID
DYNATRACE_OAUTH_CLIENT_ID = "XXXXX"
# OAUTH_CLIENT_SECRET（文字列、例: dt0s02.SAMPLE.abcd1234）- Dynatrace OAuth クライアントシークレット
DYNATRACE_OAUTH_CLIENT_SECRET = "XXXXXXX"
DYNATRACE_OAUTH_DISCOVERY_URL = "https://sso.dynatrace.com/.well-known/openid-configuration"

# AWS API 用にスコープ文字列をリストに変換
DYNATRACE_OAUTH_SCOPES = [
    "mcp-gateway:servers:invoke",
    "mcp-gateway:servers:read",
    "davis-copilot:dql2nl:execute",
    "davis-copilot:nl2dql:execute",
    "davis-copilot:conversations:execute",
    "storage:buckets:read",
    "storage:bucket-definitions:write",
    "storage:spans:read",
    "storage:entities:read",
    "storage:user.events:read",
    "storage:user.sessions:read"
]

## Gateway が引き受ける IAM ロールの作成

In [ ]:
agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("dynatrace-mcpgateway")
print("AgentCoreゲートウェイロールARN: ", agentcore_gateway_iam_role['Role']['Arn'])

## Gateway へのインバウンド認可のための Amazon Cognito プールの作成

In [ ]:
# Cognito ユーザープールを作成
import os
import boto3

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "dynatrace-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "dynatrace-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "dynatrace-agentcore-gateway-name"
CLIENT_NAME = "dynatrace-agentcore-gateway-client"
SCOPES = [
    {"ScopeName": "invoke",  # 'invoke' のみ。resource_server_id/invoke の形式でフォーマットされます
    "ScopeDescription": "AgentCore Gateway を呼び出すためのスコープ"},
]

scope_names = [f"{RESOURCE_SERVER_ID}/{scope['ScopeName']}" for scope in SCOPES]
scopeString = " ".join(scope_names)


cognito = boto3.client("cognito-idp", region_name=REGION)

print("Cognitoリソースを作成または取得中...")
gw_user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"ユーザープールID: {gw_user_pool_id}")

utils.get_or_create_resource_server(cognito, gw_user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("リソースサーバーを確認しました。")

gw_client_id, gw_client_secret = utils.get_or_create_m2m_client(cognito, gw_user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID, scope_names)

print(f"クライアントID: {gw_client_id}")

# Discovery URL を取得
gw_cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{gw_user_pool_id}/.well-known/openid-configuration'
print(gw_cognito_discovery_url)

## Gateway の作成

In [ ]:
# Cognito オーソライザーを使用して Gateway を作成。前のステップで作成した Cognito ユーザープールを使用
import boto3
gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [gw_client_id],  # クライアントは Cognito で設定された ClientId と一致する必要があります。例: 7rfbikfsm51j2fpaggacgng84g
        "discoveryUrl": gw_cognito_discovery_url
    }
}
create_response = gateway_client.create_gateway(name='ac-gateway-dynatrace-mcp',
    roleArn=agentcore_gateway_iam_role['Role']['Arn'], # IAM ロールには Gateway の作成/一覧取得/取得/削除の権限が必要
    protocolType='MCP',
    protocolConfiguration={
        'mcp': {
            'supportedVersions': ['2025-03-26'],
            'searchType': 'SEMANTIC'
        }
    },
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config,
    description='AgentCore Gateway with Dynatrace MCP Server target'
)
print(create_response)
# GatewayTarget 作成に使用する GatewayID を取得
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)

## Gateway が ACTIVE になるまで待機

In [ ]:
import time

print("ゲートウェイがACTIVEになるまで待機中...")
while True:
    response = gateway_client.get_gateway(gatewayIdentifier=gatewayID)
    status = response['status']
    print(f"ゲートウェイのステータス: {status}")
    
    if status == 'READY':
        print("ゲートウェイがREADYになりました！")
        break
    elif status == 'FAILED':
        print("ゲートウェイの作成に失敗しました！")
        break
    
    time.sleep(10)

## AgentCore Gateway のターゲットとして Dynatrace MCP Server を作成

### アウトバウンド認証の設定

まず、AgentCore Gateway が Dynatrace MCP サーバーへのアウトバウンド認証として使用する AgentCore Identity Resource Credential Provider を作成します。

In [ ]:
import boto3
identity_client = boto3.client('bedrock-agentcore-control', region_name=REGION)

dynatrace_provider = identity_client.create_oauth2_credential_provider(
    name="ac-gateway-dynatrace-mcp-identity",
    credentialProviderVendor="CustomOauth2",
    oauth2ProviderConfigInput={
        'customOauth2ProviderConfig': {
            'oauthDiscovery': {
                'discoveryUrl': DYNATRACE_OAUTH_DISCOVERY_URL,
            },
            'clientId': DYNATRACE_OAUTH_CLIENT_ID,
            'clientSecret': DYNATRACE_OAUTH_CLIENT_SECRET
        }
    }
)
dynatrace_provider_arn = dynatrace_provider['credentialProviderArn']
print(dynatrace_provider_arn)

### Gateway ターゲットの作成

In [ ]:
import boto3
gateway_client = boto3.client('bedrock-agentcore-control', region_name=REGION)
create_gateway_target_response = gateway_client.create_gateway_target(
    name='dynatrace-mcp-server-target',
    gatewayIdentifier=gatewayID,
    targetConfiguration={
        'mcp': {
            'mcpServer': {
                'endpoint': DYNATRACE_MCP_SERVER_URL
            }
        }
    },
    credentialProviderConfigurations=[
        {
            'credentialProviderType': 'OAUTH',
            'credentialProvider': {
                'oauthCredentialProvider': {
                    'providerArn': dynatrace_provider_arn,
                    'scopes': DYNATRACE_OAUTH_SCOPES
                }
            }
        },
    ]
)
print(create_gateway_target_response)

#### Gateway ターゲットが存在し、READY であることを確認

In [ ]:
list_targets_response = gateway_client.list_gateway_targets(gatewayIdentifier=gatewayID)
print(list_targets_response)

#### インバウンド認可のために Amazon Cognito からアクセストークンをリクエスト

In [ ]:
print("Amazon Cognito認証オーソライザーからアクセストークンをリクエスト中...ドメイン名の伝播が完了するまでしばらく失敗する場合があります")
token_response = utils.get_token(gw_user_pool_id, gw_client_id, gw_client_secret, scopeString, REGION)
token = token_response["access_token"]
print("トークンレスポンス:", token)

## Gateway を介して Dynatrace MCP Server ツールを検索

In [ ]:
import requests
import json

def search_tools(gateway_url, access_token, query):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }

    payload = {
        "jsonrpc": "2.0",
        "id": "search-tools-request",
        "method": "tools/call",
        "params": {
            "name": "x_amz_bedrock_agentcore_search",
            "arguments": {
                "query": query
            }
        }
    }

    response = requests.post(gateway_url, headers=headers, json=payload)
    return response.json()

# 使用例
token_response = utils.get_token(gw_user_pool_id, gw_client_id, gw_client_secret, scopeString, REGION)
access_token = token_response['access_token']
results = search_tools(gatewayURL, access_token, "problems")
print(json.dumps(results, indent=2))

## エージェントを使用して Gateway 経由で Dynatrace を操作

In [ ]:
import json

import requests
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent


def get_token():
    token = utils.get_token(gw_user_pool_id, gw_client_id, gw_client_secret, scopeString, REGION)
    return token['access_token']


def create_streamable_http_transport():
    return streamablehttp_client(
        gatewayURL, headers={"Authorization": f"Bearer {get_token()}"}
    )


client = MCPClient(create_streamable_http_transport)

## ~/.aws/credentials で設定された IAM グループ/ユーザーは Bedrock モデルへのアクセス権が必要
yourmodel = BedrockModel(
    model_id="us.anthropic.claude-sonnet-4-20250514-v1:0", # リージョンによって model_id の更新が必要な場合があります
    temperature=0.7,
    max_tokens=500,  # レスポンスの長さを制限
)

with client:
    # listTools を呼び出し
    tools = client.list_tools_sync()
    # モデルとツールを使用してエージェントを作成
    agent = Agent(
        model=yourmodel, tools=tools
    )  ## 任意のモデルに置き換え可能
    # サンプルプロンプトでエージェントを呼び出し。これは MCP listTools を呼び出し、LLM がアクセスできるツールのリストを取得するだけです。以下は実際にはツールを呼び出しません。
    agent("Hi, can you list all tools available to you")
    # 簡略化されたプロンプトとエラー処理
    result = agent("Show me top vulnerabilities in my Dynatrace environment")

## クリーンアップ
IAM ロール、IAM ポリシー、認証情報プロバイダー、Cognito ユーザープールなどの追加リソースも作成されており、クリーンアップの一環として手動で削除する必要がある場合があります。

### Gateway の削除（オプション）

In [ ]:
utils.delete_gateway(gateway_client, gatewayID)

### Identity Provider の削除（オプション）

In [ ]:
identity_client.delete_oauth2_credential_provider(name='ac-gateway-dynatrace-mcp-identity')